In [1]:
import pandas as pd

In [22]:
df = pd.read_csv("Data/index_archieve/security.csv", skiprows=2, encoding='cp1251', sep=";")

In [28]:
df.describe()

,DURATION,YIELD,DECIMALS,TRADINGSESSION,VOLUME
count,1517.0,1517.0,5489.0,5489.0,0.0
mean,0.0,0.0,2.0,3.0,NaN
std,0.0,0.0,0.0,0.0,NaN
min,0.0,0.0,2.0,3.0,NaN
25%,0.0,0.0,2.0,3.0,NaN
50%,0.0,0.0,2.0,3.0,NaN
75%,0.0,0.0,2.0,3.0,NaN
max,0.0,0.0,2.0,3.0,NaN


In [ ]:
df['TRADEDATE'] = pd.to_datetime(df['TRADEDATE'])